[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/inferencebot-colab/blob/main/inferencebot_colab.ipynb)

In [ ]:
%cd /content

!wget https://go.dev/dl/go1.21.5.linux-amd64.tar.gz
!tar -C /content -xzf go1.21.5.linux-amd64.tar.gz
import os
os.environ["PATH"] = os.environ.get("PATH", "") + ":/content/go/bin"
os.environ["GOPATH"] = os.path.expanduser("~/go")
os.environ["PATH"] = os.environ.get("PATH", "") + ":" + os.path.join(os.environ["GOPATH"], "bin")
!wget https://github.com/camenduru/inferencebot/raw/main/index.html -O /content/index.html
!wget https://github.com/camenduru/inferencebot/raw/main/main.go -O /content/main.go
!go mod init github.com/camenduru/websocket-echo-server
!go get github.com/gorilla/websocket
!go get github.com/gempir/go-twitch-irc/v3
!go get github.com/mvdan/xurls

In [ ]:
#@title stream-diffusion

!pip install -q git+https://github.com/cumulo-autumn/StreamDiffusion.git@main#egg=streamdiffusion
!python -m streamdiffusion.tools.install-tensorrt
!pip install -q accelerate gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl

import torch
from diffusers import AutoencoderTiny, StableDiffusionPipeline

from streamdiffusion import StreamDiffusion
from streamdiffusion.image_utils import postprocess_image

pipe = StableDiffusionPipeline.from_pretrained("KBlueLeaf/kohaku-v2.1").to(
    device=torch.device("cuda"),
    dtype=torch.float16,
)

stream = StreamDiffusion(
    pipe,
    t_index_list=[0, 16, 32, 45],
    torch_dtype=torch.float16,
    cfg_type="none",
)

stream.load_lcm_lora()
stream.fuse_lora()
stream.vae = AutoencoderTiny.from_pretrained("madebyollin/taesd").to(device=pipe.device, dtype=pipe.dtype)
pipe.enable_xformers_memory_efficient_attention()

import gradio as gr

def generate(prompt):
  stream.prepare(prompt)
  for _ in range(4):
    stream()
  x_output = stream.txt2img()
  image = postprocess_image(x_output, output_type="pil")[0]
  image.save('/content/image.jpg')
  return image.resize((512, 512))

with gr.Blocks(title=f"Realtime SDXL Turbo", css=".gradio-container {max-width: 544px !important}") as demo:
    with gr.Row():
      with gr.Column():
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")
          button = gr.Button()
    with gr.Row(variant="default"):
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=512,
            width=512,
            elem_id="output_image",
        )
    button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=False)

demo.queue().launch(inline=False, share=True, debug=False)

In [ ]:
#@title sdxl-turbo

!pip install -q diffusers transformers accelerate peft gradio==3.50.2

from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16").to("cuda")

import gradio as gr

def generate(prompt):
  image = pipe(prompt, num_inference_steps=1, guidance_scale=0.0, width=512, height=512).images[0]
  image.save('/content/image.jpg')
  return image.resize((512, 512))

with gr.Blocks(title=f"Realtime SDXL Turbo", css=".gradio-container {max-width: 544px !important}") as demo:
    with gr.Row():
      with gr.Column():
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")
          button = gr.Button()
    with gr.Row(variant="default"):
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=512,
            width=512,
            elem_id="output_image",
        )

    button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=False)

demo.queue().launch(inline=False, share=True, debug=False)

In [ ]:
username = "username_sed" #@param {type: 'string'}
username_code = f's/username_sed/{username}/g'
channel = "channel_sed" #@param {type: 'string'}
channel_code = f's/channel_sed/{channel}/g'
oauth = "oauth_sed" #@param {type: 'string'}
oauth_code = f's/oauth_sed/{oauth}/g'
!sed -i -e {username_code} -e {channel_code} -e {oauth_code} main.go

import os
import subprocess
import threading

def set_environment_variables():
    os.environ["PATH"] = os.environ.get("PATH", "") + ":/content/go/bin"
    os.environ["GOPATH"] = os.path.expanduser("~/go")
    os.environ["PATH"] = os.environ.get("PATH", "") + ":" + os.path.join(os.environ["GOPATH"], "bin")

def run_go_program():
    try:
        subprocess.run(["go", "run", "."], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Go program: {e}")

def main():
    set_environment_variables()
    go_thread = threading.Thread(target=run_go_program)
    go_thread.start()
    # go_thread.join()

if __name__ == "__main__":
    main()